# Final Capstone Project
## Analyze the Neighborhood in Greater Cincinnati Metropolitan Area in the state of Ohio

### Introduction:
##### **Objective:** The project is to analyze the main cities within the Greater Cincinnati Area of Ohio (OH) and decide on the best market to launch a new Asian Grocery Supermarket.
##### The client wants to explore the cities in terms of **Restaurants** and **Schools** to get an idea of Asian population in these cities. School analysis presumes the current and future immigrant Asian population may be more focused on cities with high standard of education.

##### To limit the scope of work for this PoC we will assume the new store will be primarily targeting Indian, Chinese and other Asian population.

##### So we will work to present each city in terms of number of Indian/Chinese/Asian Restaurants, schools and school district details. 

### Here is a summary of all the key steps involved in analysis
#### 1. Gather the Cities and visualize their locations:
##### This step reads the CSV file for the information on cities (Name and Geo Coordinates) within Greater Cincinnati and view them on the map to get an good view of their relative locations using FOLIUM.
##### We have explored the wiki pages and made a summary of main cities of Greater Cincinnati Area with Ohio. The pages are so spread that it made more sense to do some work offline and gather the information in an csv rather than web scrapping. We will use that CSV file to read this first information. Here is the Wiki page for reference.
###### <WikiPage


#### 2. Gather Restaurant Data:
##### We are using FOURSQUARE APIs to get the Restaurants information in those cities.

##### 2.a) Start with exploring one city: 
###### - We will explore one city first to understand FOURSQUARE results.

##### 2.b) Gather Restaurants data for all the cities in question
###### - Now repeat the process using a new function to get the venues for all the Greater Cincinnati cities that we are checking

#### 3. Gather Schools Data:
#####  Again using FOURSQUARE APIs we will get schools information in those cities
##### 3.a) Explore the schools data for one city like we did for restaurants
##### 3.b) Get schools locations data for the cities using Foursquare APIs.
##### 3.c) Also get Schools demographic and performance data (grades, population etc.) available at the state education departments sites.
###### - This information is also little spread across multiple sites, so we did some off line work to consolidate the information for easy analysis.
###### - State educational site pages                
                 
#### 4) Combine all information for final analysis and presentation
##### 4.a) combine schools location data (3a) with restaurants location data (2) and group them for Citywise representation (one row for each city)
##### 4.b) Then add schools grade and population information (3.b) as well for each city
###### NOTE: Few of the cities may be covered by multiple school districts (SD), in such cases we have taken one SD that represents best.
                 
#### 5) Final Analysis and Results
##### So here our final data is ready and presentable. 
##### For our requirement that is to *"find the best market to launch the new Asian Grocery Supermarket"*, let us use some technique to rank the cities. 
##### We will assign scores (Index) for each information criteria gathered, and total that to come with the final index for this city. For simplification, let us use just the total value of each criteria (Restaurants Total, Schools Total, Enrollments Total).
                                  
#### 6) Conclude: Summarize
##### Some conclusive remarks on the analysis and how further work can be done to make the analysis more exhaustive as required to cover more requirements and use cases

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
r = requests.get('https://en.wikipedia.org/wiki/Cincinnati_metropolitan_area')

#### 1. Gather the Cities and visualize their locations:
##### This step reads the CSV file for the information on cities (Name and Geo Coordinates) within Greater Cincinnati and view them on the map to get an good view of their relative locations using FOLIUM.

##### NOTE: LEA stands for "Local Education Agency". LEAId is the unique identifier of a School District. With every city we have tagged a School District that fits it best.

In [3]:
gcincities = pd.read_csv("Manual_Data_Prep.csv")
gcincities.head(3)

,cityname,population,LEAId
0,"Cincinnati, Ohio","298,843",43752
1,"Hamilton, Butler County, OH","62,447",44107
2,"Liberty Township, Butler County, Ohio,37","259""",46110


##### Now get the Latitude Longitude for the cities using geolocator

In [4]:
latlong = []
for city in gcincities['cityname']:
    address = city
    geolocator = Nominatim(user_agent="my-application")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
    latlong.append((latitude, longitude))
#latlong

The geograpical coordinate of Cincinnati, Ohio are 39.1014537, -84.5124602.
The geograpical coordinate of Hamilton, Butler County, OH are 39.3995067, -84.5613428.
The geograpical coordinate of Liberty Township, Butler County, Ohio,37 are 39.4015584, -84.4096782418939.
The geograpical coordinate of Middletown, Butler County, Ohio are 39.5150576, -84.3982763.
The geograpical coordinate of Fairfield, Butler, Ohio are 39.3458953, -84.5605031.
The geograpical coordinate of Oxford, Butler County, Ohio are 39.5103048, -84.7420519.
The geograpical coordinate of Mason, Warren County, Ohio are 39.3600586, -84.309939.
The geograpical coordinate of Lebanon, Warren County, Ohio are 39.4353373, -84.2029922.
The geograpical coordinate of South Lebanon, Warren County, Ohio are 39.3710358, -84.2128058.
The geograpical coordinate of Springboro, Warren County, Ohio are 39.5522815, -84.2332718.
The geograpical coordinate of Norwood, Hamilton County, Ohio are 39.1556149, -84.4596641.
The geograpical coordi

##### Store the data in a dataframe

In [5]:
df = pd.DataFrame(latlong, columns=['Latitude', 'Longitude']) 
df.head()

,Latitude,Longitude
0,39.101454,-84.512460
1,39.399507,-84.561343
2,39.401558,-84.409678
3,39.515058,-84.398276
4,39.345895,-84.560503


##### Add this locations data along with cities in the main dataframe.

In [6]:
gcincities[['Latitude','Longitude']] = df[['Latitude','Longitude']]
gcincities.head()

,cityname,population,LEAId,Latitude,Longitude
0,"Cincinnati, Ohio","298,843",43752,39.101454,-84.512460
1,"Hamilton, Butler County, OH","62,447",44107,39.399507,-84.561343
2,"Liberty Township, Butler County, Ohio,37","259""",46110,39.401558,-84.409678
3,"Middletown, Butler County, Ohio","48,694",44404,39.515058,-84.398276
4,"Fairfield, Butler, Ohio","42,510",46102,39.345895,-84.560503


In [7]:
print('The dataframe has {} cities.'.format(gcincities.shape[0]))

The dataframe has 37 cities.


##### Let us use Cincinnati downtown as the base location to create the Folium map

In [8]:
address = 'Cincinnati, Ohio'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Cincinnati are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Cincinnati are 39.1014537, -84.5124602.


##### Create FOLIUM map to view all the cities on the map

In [9]:
# create map of New York using latitude and longitude values
map_gtcincinnati = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city in zip(gcincities['Latitude'], gcincities['Longitude'], gcincities['cityname']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_gtcincinnati)  
    
map_gtcincinnati

In [10]:
# Here reduce the number of cities if needed, based on population, or state (OH)

#### 2. Gather Restaurant Data:
##### We are using FOURSQUARE APIs to get the Restaurants information in those cities.

##### Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = 'AYQU5RSQ34C21TJO55C5BVXYVMVCNMZMHEUJR0T1UUUIQ4JT' # your Foursquare ID
CLIENT_SECRET = 'VGFDQ0BLHPZU3YUPBCYRG2KXBJLU3SMOAAFNAZDP5GGZW0HU' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AYQU5RSQ34C21TJO55C5BVXYVMVCNMZMHEUJR0T1UUUIQ4JT
CLIENT_SECRET:VGFDQ0BLHPZU3YUPBCYRG2KXBJLU3SMOAAFNAZDP5GGZW0HU


##### 2.a) Start with exploring one city: 
##### - We will explore one city first to understand FOURSQUARE results.

##### Get the neighborhood's name, lat, long

In [12]:
gcincity_latitude = gcincities.loc[0, 'Latitude'] # neighborhood latitude value
gcincity_longitude = gcincities.loc[0, 'Longitude'] # neighborhood longitude value

gcincity_name = gcincities.loc[0, 'cityname'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(gcincity_name, 
                                                               gcincity_latitude, 
                                                               gcincity_longitude))

Latitude and longitude values of Cincinnati, Ohio are 39.1014537, -84.5124602.


##### Now, let's get the top 100 venues that are in this city within a radius of 8500 meters, that is like 5 miles.
##### First, let's create the GET request URL. Name your URL url.

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 8500 # define radius

search_query = 'Indian'

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&query={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION,
    search_query,
    gcincity_latitude, 
    gcincity_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AYQU5RSQ34C21TJO55C5BVXYVMVCNMZMHEUJR0T1UUUIQ4JT&client_secret=VGFDQ0BLHPZU3YUPBCYRG2KXBJLU3SMOAAFNAZDP5GGZW0HU&v=20180605&query=Indian&ll=39.1014537,-84.5124602&radius=8500&limit=100'

##### Send the GET request and examine the resutls

In [14]:
results = requests.get(url).json()
#results

##### The below get_category_type function helps us handle the exceptions where the Category of a particular venue is not available in Foursquare. 'None' is returned in such cases

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

##### Now we clean the json and structure it into a pandas dataframe.

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON
#nearby_venues

In [17]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Ambar India,Indian Restaurant,39.143944,-84.521502
1,Krishna,Indian Restaurant,39.128515,-84.518813
2,Deep India,Indian Restaurant,39.127878,-84.519894
3,Dusmesh,Indian Restaurant,39.152084,-84.534534
4,Guru India Restaurant,Indian Restaurant,39.049111,-84.574491


In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


In [19]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
Ethiopian Restaurant,1,1,1
Food,2,2,2
Grocery Store,2,2,2
Indian Chinese Restaurant,1,1,1
Indian Restaurant,15,15,15
South Indian Restaurant,1,1,1


In [20]:
print('There are {} uniques categories returned for the sample city that we checked'.format(len(nearby_venues['categories'].unique())))

There are 6 uniques categories returned for the sample city that we checked


In [21]:
#nearby_venues.loc[nearby_venues['categories'] == 'Indian Restaurant']

#### 2.b) Gather Restaurants data for all the cities in question
##### - Now repeat the process using a new function to get the venues for all the Greater Cincinnati cities that we are 

In [22]:
# This function takes Cities details, Category Type, and Radius range as inputs and returns the venues
def getNearbyVenues(names, latitudes, longitudes, category, radius):
    LIMIT=100
    venues_list=[]
    venues_list2=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&query={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            category,
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        for v2 in results:
            if len(v2['venue']['categories']) == 0:
                category2 = 'None'
            else:
                category2 = v2['venue']['categories'][0]['name']
            
            venues_list2.append((name, 
            lat, 
            lng, 
            v2['venue']['name'], 
            v2['venue']['location']['lat'], 
            v2['venue']['location']['lng'],  
            category2))
   
    nearby_venues2 = pd.DataFrame([item for item in venues_list2])
    nearby_venues2.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue Name', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues2)

##### Get Restaurants by Category by Categories: This is because the Foursquare API returns max 100 venues in one call. So we have to break the API calls in parts (Indian/Chinese) so as to get maximum useful data.

In [23]:
# Execute the above function, to get Indian venues for all the cities at once
gcincities_venues_in = getNearbyVenues(names=gcincities['cityname'],
                                   latitudes=gcincities['Latitude'],
                                   longitudes=gcincities['Longitude'],
                                   category='Indian',
                                   radius=8500
                                  )

Cincinnati, Ohio
Hamilton, Butler County, OH
Liberty Township, Butler County, Ohio,37
Middletown, Butler County, Ohio
Fairfield, Butler, Ohio
Oxford, Butler County, Ohio
Mason, Warren County, Ohio
Lebanon, Warren County, Ohio
South Lebanon, Warren County, Ohio
Springboro, Warren County, Ohio
Norwood, Hamilton County, Ohio
Forest Park, Hamilton County, Ohio
Sharonville, Hamilton County, Ohio
Blue Ash, Hamilton County, Ohio
Wilmington, Clinton County, Ohio
Loveland, Hamilton County, Ohio
Springdale, Hamilton County, Ohio
Indian Hill, Ohio
Franklin, Warren County, Ohio
Reading, Hamilton County, Ohio
St. Bernard, Hamilton County, Ohio
Delhi Township, Hamilton County, Ohio
Mount Healthy, Hamilton County, Ohio
College Hill, Hamilton County, Ohio
Westwood, Hamilton County, Ohio
Milford, Clermont County, Ohio
Batavia, Clermont County, Ohio
West Chester, Butler County, Ohio
Trenton, Butler County, Ohio
Mount Orab, Brown County, Ohio
Blanchester, Ohio
Georgetown, Ohio
Amelia, Clermont County, Oh

In [24]:
gcincities_venues_in.shape

(349, 7)

In [25]:
# Execute the above function, to get Chinese venues for all the cities at once
gcincities_venues_ch = getNearbyVenues(names=gcincities['cityname'],
                                   latitudes=gcincities['Latitude'],
                                   longitudes=gcincities['Longitude'],
                                   category='Chinese',
                                   radius=8500
                                  )

Cincinnati, Ohio
Hamilton, Butler County, OH
Liberty Township, Butler County, Ohio,37
Middletown, Butler County, Ohio
Fairfield, Butler, Ohio
Oxford, Butler County, Ohio
Mason, Warren County, Ohio
Lebanon, Warren County, Ohio
South Lebanon, Warren County, Ohio
Springboro, Warren County, Ohio
Norwood, Hamilton County, Ohio
Forest Park, Hamilton County, Ohio
Sharonville, Hamilton County, Ohio
Blue Ash, Hamilton County, Ohio
Wilmington, Clinton County, Ohio
Loveland, Hamilton County, Ohio
Springdale, Hamilton County, Ohio
Indian Hill, Ohio
Franklin, Warren County, Ohio
Reading, Hamilton County, Ohio
St. Bernard, Hamilton County, Ohio
Delhi Township, Hamilton County, Ohio
Mount Healthy, Hamilton County, Ohio
College Hill, Hamilton County, Ohio
Westwood, Hamilton County, Ohio
Milford, Clermont County, Ohio
Batavia, Clermont County, Ohio
West Chester, Butler County, Ohio
Trenton, Butler County, Ohio
Mount Orab, Brown County, Ohio
Blanchester, Ohio
Georgetown, Ohio
Amelia, Clermont County, Oh

In [26]:
gcincities_venues_ch.shape

(558, 7)

In [27]:
#gcincities_venues_ch.loc[gcincities_venues_ch['City'] == 'Mason, Ohio']

##### Combine the different categories of Restaurants

In [28]:
gcincities_venues = gcincities_venues_in.append(gcincities_venues_ch, ignore_index=True)

##### Check the size of the resulting dataframe

In [29]:
print(gcincities_venues.shape)
gcincities_venues.head()

(907, 7)


,City,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,"Cincinnati, Ohio",39.101454,-84.51246,Ambar India,39.143944,-84.521502,Indian Restaurant
1,"Cincinnati, Ohio",39.101454,-84.51246,Krishna,39.128515,-84.518813,Indian Restaurant
2,"Cincinnati, Ohio",39.101454,-84.51246,Deep India,39.127878,-84.519894,Indian Restaurant
3,"Cincinnati, Ohio",39.101454,-84.51246,Dusmesh,39.152084,-84.534534,Indian Restaurant
4,"Cincinnati, Ohio",39.101454,-84.51246,Guru India Restaurant,39.049111,-84.574491,Indian Restaurant


##### Let's check how many venues were returned for each neighborhood

In [30]:
gcincities_venues.groupby('City').count()

,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
"Amelia, Clermont County, Ohio",8,8,8,8,8,8
"Batavia, Clermont County, Ohio",5,5,5,5,5,5
"Bethel, Clermont County, Ohio",1,1,1,1,1,1
"Blanchester, Ohio",2,2,2,2,2,2
"Blue Ash, Hamilton County, Ohio",54,54,54,54,54,54
"Cincinnati, Ohio",65,65,65,65,65,65
"College Hill, Hamilton County, Ohio",31,31,31,31,31,31
"Delhi Township, Hamilton County, Ohio",25,25,25,25,25,25
"Fairfield, Butler, Ohio",18,18,18,18,18,18


##### Let's check Categories as well as count of venues for each Category

In [31]:
gcincities_venues.groupby('Venue Category').count()

,City,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Asian Restaurant,24,24,24,24,24,24
Athletics & Sports,1,1,1,1,1,1
Bar,6,6,6,6,6,6
Bubble Tea Shop,4,4,4,4,4,4
Burger Joint,3,3,3,3,3,3
Cantonese Restaurant,3,3,3,3,3,3
Chinese Restaurant,485,485,485,485,485,485
Church,8,8,8,8,8,8
City,9,9,9,9,9,9


##### Let's find out how many unique categories can be curated from all the returned venues

In [32]:
#print('There are {} uniques categories.'.format(len(gcincities_venues['Venue Category'].unique())))

##### Looks like lot of noise, some not-relevant categories in the result. Let us keep only the categories of interest (Indian/Chinese/Asian Restaurants)
##### Copy the relevant rows in a new dataframe that will be used for all further analysis

In [57]:
checkrest = ['Chinese','Indian','Indian Restaurant','South Indian Restaurant','North Indian Restaurant','Indian Chinese Restaurant','Chinese Restaurant','Dim Sum Restaurant','Japanese Restaurant','Shanghai Restaurant','Asian Restaurant','Thai Restaurant','Vietnamese Restaurant']
gcincities_venues_data = gcincities_venues.loc[gcincities_venues['Venue Category'].isin(checkrest)]

In [58]:
gcincities_venues_data.shape

(749, 7)

##### Repeat the step again to check Categories as well as count of venues for each Category

In [59]:
gcincities_venues_data.groupby('Venue Category').count()

,City,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Asian Restaurant,24,24,24,24,24,24
Chinese Restaurant,485,485,485,485,485,485
Dim Sum Restaurant,6,6,6,6,6,6
Indian Chinese Restaurant,3,3,3,3,3,3
Indian Restaurant,200,200,200,200,200,200
Japanese Restaurant,6,6,6,6,6,6
North Indian Restaurant,3,3,3,3,3,3
Shanghai Restaurant,4,4,4,4,4,4
South Indian Restaurant,11,11,11,11,11,11


##### Let us keep only the main Categories as explicit categories (Indian/Chinese), and group all others as a common category 'Asian Restaurants'

In [60]:
def clean(x):
   # print(x['Venue Category'])
    if x['Venue Category'].find('Indian') != -1:
       retval = 'Indian Restaurant'
    elif x['Venue Category'].find('Chinese') != -1:
       retval = 'Chinese Restaurant'
    else:
       retval = 'Other Asian Restaurant'
   # print(retval)
    return retval

In [61]:
gcincities_venues_data.loc[0]

City                Cincinnati, Ohio
City Latitude                39.1015
City Longitude              -84.5125
Venue Name               Ambar India
Venue Latitude               39.1439
Venue Longitude             -84.5215
Venue Category     Indian Restaurant
Name: 0, dtype: object

In [62]:
gcincities_venues_data.loc[:,'Venue Category2'] = gcincities_venues_data.apply (lambda row: clean (row),axis=1)

In [63]:
gcincities_venues_data.groupby('Venue Category2').count()

,City,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
Venue Category2,,,,,,,
Chinese Restaurant,485,485,485,485,485,485,485
Indian Restaurant,217,217,217,217,217,217,217
Other Asian Restaurant,47,47,47,47,47,47,47


In [64]:
gcincities_venues_data.drop(['Venue Category'], inplace=True, axis=1)

In [65]:
gcincities_venues_data.rename(columns={'Venue Category2':'Venue Category'}, inplace=True)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [66]:
gcincities_venues_data.head()

,City,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,"Cincinnati, Ohio",39.101454,-84.51246,Ambar India,39.143944,-84.521502,Indian Restaurant
1,"Cincinnati, Ohio",39.101454,-84.51246,Krishna,39.128515,-84.518813,Indian Restaurant
2,"Cincinnati, Ohio",39.101454,-84.51246,Deep India,39.127878,-84.519894,Indian Restaurant
3,"Cincinnati, Ohio",39.101454,-84.51246,Dusmesh,39.152084,-84.534534,Indian Restaurant
4,"Cincinnati, Ohio",39.101454,-84.51246,Guru India Restaurant,39.049111,-84.574491,Indian Restaurant


In [67]:
#Also just check thee Citywise count of venues
gcincities_venues_data.groupby('City').count()

,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
"Amelia, Clermont County, Ohio",7,7,7,7,7,7
"Batavia, Clermont County, Ohio",4,4,4,4,4,4
"Blanchester, Ohio",2,2,2,2,2,2
"Blue Ash, Hamilton County, Ohio",39,39,39,39,39,39
"Cincinnati, Ohio",57,57,57,57,57,57
"College Hill, Hamilton County, Ohio",27,27,27,27,27,27
"Delhi Township, Hamilton County, Ohio",23,23,23,23,23,23
"Fairfield, Butler, Ohio",14,14,14,14,14,14
"Forest Park, Hamilton County, Ohio",28,28,28,28,28,28


##### For our analysis, we may want to already filter out the cities that have less than 5 venues. We will likely not choose that for new business

In [68]:
gcincities_venues_data1 = gcincities_venues_data.groupby('City').filter(lambda group: len(group) >= 5)

In [69]:
gcincities_venues_data1.groupby('City').count()

,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
"Amelia, Clermont County, Ohio",7,7,7,7,7,7
"Blue Ash, Hamilton County, Ohio",39,39,39,39,39,39
"Cincinnati, Ohio",57,57,57,57,57,57
"College Hill, Hamilton County, Ohio",27,27,27,27,27,27
"Delhi Township, Hamilton County, Ohio",23,23,23,23,23,23
"Fairfield, Butler, Ohio",14,14,14,14,14,14
"Forest Park, Hamilton County, Ohio",28,28,28,28,28,28
"Franklin, Warren County, Ohio",7,7,7,7,7,7
"Hamilton, Butler County, OH",10,10,10,10,10,10


##### Great, Now also drop those cities (<5 Rest Count) from the main dataset to avoid futher processing of those cities, to save resources
##### First let us get the names of the *cities to be retained* in a list

In [70]:
gcincities_venues_ret_city_list = list(gcincities_venues_data1['City'].unique())
gcincities_venues_ret_city_list

['Cincinnati, Ohio',
 'Liberty Township, Butler County, Ohio,37',
 'Oxford, Butler County, Ohio',
 'Mason, Warren County, Ohio',
 'South Lebanon, Warren County, Ohio',
 'Springboro, Warren County, Ohio',
 'Norwood, Hamilton County, Ohio',
 'Forest Park, Hamilton County, Ohio',
 'Sharonville, Hamilton County, Ohio',
 'Blue Ash, Hamilton County, Ohio',
 'Loveland, Hamilton County, Ohio',
 'Springdale, Hamilton County, Ohio',
 'Indian Hill, Ohio',
 'Reading, Hamilton County, Ohio',
 'St. Bernard, Hamilton County, Ohio',
 'Delhi Township, Hamilton County, Ohio',
 'Mount Healthy, Hamilton County, Ohio',
 'College Hill, Hamilton County, Ohio',
 'Westwood, Hamilton County, Ohio',
 'Milford, Clermont County, Ohio',
 'West Chester, Butler County, Ohio',
 'Amelia, Clermont County, Ohio',
 'Hamilton, Butler County, OH',
 'Middletown, Butler County, Ohio',
 'Fairfield, Butler, Ohio',
 'Franklin, Warren County, Ohio',
 'Trenton, Butler County, Ohio']

In [71]:
gcincities_rem = gcincities[~gcincities['cityname'].isin(gcincities_venues_ret_city_list)]
print('The cities to be removed are')
gcincities_rem

The cities to be removed are


,cityname,population,LEAId,Latitude,Longitude
7,"Lebanon, Warren County, Ohio","20,033",44214,39.435337,-84.202992
14,"Wilmington, Clinton County, Ohio","12,520",45112,39.445339,-83.828537
26,"Batavia, Clermont County, Ohio","1,509",46300,39.077007,-84.176879
29,"Mount Orab, Brown County, Ohio","3,664",46060,39.027566,-83.919652
30,"Blanchester, Ohio","4,243",46383,39.293117,-83.988821
31,"Georgetown, Ohio","4,331",45377,38.864514,-83.904094
33,"Williamsburg, Clermont County, Ohio","2,490",46367,39.054231,-84.052989
34,"New Richmond, Clermont County, Ohio","2,582",45559,38.948676,-84.279937
35,"Bethel, Clermont County, Ohio","2,711",46318,38.963677,-84.080766
36,"Fayetteville, Brown County, Ohio",330,46045,39.186730,-83.932152


In [72]:
gcincities_retained = gcincities[gcincities['cityname'].isin(gcincities_venues_ret_city_list)]
print('The cities retained are')
gcincities_retained

The cities retained are


,cityname,population,LEAId,Latitude,Longitude
0,"Cincinnati, Ohio","298,843",43752,39.101454,-84.512460
1,"Hamilton, Butler County, OH","62,447",44107,39.399507,-84.561343
2,"Liberty Township, Butler County, Ohio,37","259""",46110,39.401558,-84.409678
3,"Middletown, Butler County, Ohio","48,694",44404,39.515058,-84.398276
4,"Fairfield, Butler, Ohio","42,510",46102,39.345895,-84.560503
5,"Oxford, Butler County, Ohio","21,943",46151,39.510305,-84.742052
6,"Mason, Warren County, Ohio","30,712",50450,39.360059,-84.309939
8,"South Lebanon, Warren County, Ohio","4,115",50435,39.371036,-84.212806
9,"Springboro, Warren County, Ohio","17,409",50427,39.552281,-84.233272
10,"Norwood, Hamilton County, Ohio","19,207",44578,39.155615,-84.459664


In [73]:
gcincities=gcincities_retained

In [74]:
gcincities.shape

(27, 5)

### ==== 2b Outcome => gcincities(list of cities) and gcincities_venues_data1 (citywise restaurants data) ====

#### 3. Gather Schools Data:
#####  Again using FOURSQUARE APIs we will get schools information in those cities

#### 3.a) Explore the schools data for one city like we did for restaurants

In [75]:
gcincities_schools_exp = getNearbyVenues(names=gcincities['cityname'].head(1),
                                   latitudes=gcincities['Latitude'].head(1),
                                   longitudes=gcincities['Longitude'].head(1),
                                   category='School',
                                   radius=8500
                                  )

Cincinnati, Ohio


In [76]:
gcincities_schools_exp.groupby('Venue Category').count()

,City,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Building,1,1,1,1,1,1
Church,1,1,1,1,1,1
College Academic Building,1,1,1,1,1,1
College Auditorium,1,1,1,1,1,1
College Classroom,2,2,2,2,2,2
Community College,1,1,1,1,1,1
Elementary School,12,12,12,12,12,12
General College & University,1,1,1,1,1,1
High School,22,22,22,22,22,22


##### NOTE: By exploring the schools data for one City, we understood that their is lot of noise, like Restaurants data. So we do specific query for specific type of schools that is likely to get significant numbers.

#### 3.b) Get schools locations data for the cities using Foursquare APIs.
##### Again like before we break this API call in parts (Pre/Elem/Middle/High schools) to get more data.

In [77]:
# High
gcincities_schools_h = getNearbyVenues(names=gcincities['cityname'],
                                   latitudes=gcincities['Latitude'],
                                   longitudes=gcincities['Longitude'],
                                   category='High School',
                                   radius=8500
                                  )

# Middle
gcincities_schools_m = getNearbyVenues(names=gcincities['cityname'],
                                   latitudes=gcincities['Latitude'],
                                   longitudes=gcincities['Longitude'],
                                   category='Middle School',
                                   radius=8500
                                  )

Cincinnati, Ohio
Hamilton, Butler County, OH
Liberty Township, Butler County, Ohio,37
Middletown, Butler County, Ohio
Fairfield, Butler, Ohio
Oxford, Butler County, Ohio
Mason, Warren County, Ohio
South Lebanon, Warren County, Ohio
Springboro, Warren County, Ohio
Norwood, Hamilton County, Ohio
Forest Park, Hamilton County, Ohio
Sharonville, Hamilton County, Ohio
Blue Ash, Hamilton County, Ohio
Loveland, Hamilton County, Ohio
Springdale, Hamilton County, Ohio
Indian Hill, Ohio
Franklin, Warren County, Ohio
Reading, Hamilton County, Ohio
St. Bernard, Hamilton County, Ohio
Delhi Township, Hamilton County, Ohio
Mount Healthy, Hamilton County, Ohio
College Hill, Hamilton County, Ohio
Westwood, Hamilton County, Ohio
Milford, Clermont County, Ohio
West Chester, Butler County, Ohio
Trenton, Butler County, Ohio
Amelia, Clermont County, Ohio
Cincinnati, Ohio
Hamilton, Butler County, OH
Liberty Township, Butler County, Ohio,37
Middletown, Butler County, Ohio
Fairfield, Butler, Ohio
Oxford, Butler

In [78]:
#Elementary
gcincities_schools_e = getNearbyVenues(names=gcincities['cityname'],
                                   latitudes=gcincities['Latitude'],
                                   longitudes=gcincities['Longitude'],
                                   category='Elementary School',
                                   radius=8500
                                  )

# Preschools
gcincities_schools_p = getNearbyVenues(names=gcincities['cityname'],
                                   latitudes=gcincities['Latitude'],
                                   longitudes=gcincities['Longitude'],
                                   category='Preschool',
                                   radius=8500
                                  )

Cincinnati, Ohio
Hamilton, Butler County, OH
Liberty Township, Butler County, Ohio,37
Middletown, Butler County, Ohio
Fairfield, Butler, Ohio
Oxford, Butler County, Ohio
Mason, Warren County, Ohio
South Lebanon, Warren County, Ohio
Springboro, Warren County, Ohio
Norwood, Hamilton County, Ohio
Forest Park, Hamilton County, Ohio
Sharonville, Hamilton County, Ohio
Blue Ash, Hamilton County, Ohio
Loveland, Hamilton County, Ohio
Springdale, Hamilton County, Ohio
Indian Hill, Ohio
Franklin, Warren County, Ohio
Reading, Hamilton County, Ohio
St. Bernard, Hamilton County, Ohio
Delhi Township, Hamilton County, Ohio
Mount Healthy, Hamilton County, Ohio
College Hill, Hamilton County, Ohio
Westwood, Hamilton County, Ohio
Milford, Clermont County, Ohio
West Chester, Butler County, Ohio
Trenton, Butler County, Ohio
Amelia, Clermont County, Ohio
Cincinnati, Ohio
Hamilton, Butler County, OH
Liberty Township, Butler County, Ohio,37
Middletown, Butler County, Ohio
Fairfield, Butler, Ohio
Oxford, Butler

In [79]:
#Nursery
gcincities_schools_n = getNearbyVenues(names=gcincities['cityname'],
                                   latitudes=gcincities['Latitude'],
                                   longitudes=gcincities['Longitude'],
                                   category='Nursery School',
                                   radius=8500
                                  )

# Daycare
gcincities_schools_d = getNearbyVenues(names=gcincities['cityname'],
                                   latitudes=gcincities['Latitude'],
                                   longitudes=gcincities['Longitude'],
                                   category='Daycare',
                                   radius=8500
                                  )

Cincinnati, Ohio
Hamilton, Butler County, OH
Liberty Township, Butler County, Ohio,37
Middletown, Butler County, Ohio
Fairfield, Butler, Ohio
Oxford, Butler County, Ohio
Mason, Warren County, Ohio
South Lebanon, Warren County, Ohio
Springboro, Warren County, Ohio
Norwood, Hamilton County, Ohio
Forest Park, Hamilton County, Ohio
Sharonville, Hamilton County, Ohio
Blue Ash, Hamilton County, Ohio
Loveland, Hamilton County, Ohio
Springdale, Hamilton County, Ohio
Indian Hill, Ohio
Franklin, Warren County, Ohio
Reading, Hamilton County, Ohio
St. Bernard, Hamilton County, Ohio
Delhi Township, Hamilton County, Ohio
Mount Healthy, Hamilton County, Ohio
College Hill, Hamilton County, Ohio
Westwood, Hamilton County, Ohio
Milford, Clermont County, Ohio
West Chester, Butler County, Ohio
Trenton, Butler County, Ohio
Amelia, Clermont County, Ohio
Cincinnati, Ohio
Hamilton, Butler County, OH
Liberty Township, Butler County, Ohio,37
Middletown, Butler County, Ohio
Fairfield, Butler, Ohio
Oxford, Butler

##### Combine all the data in one dataframe

In [80]:
gcincities_schools = gcincities_schools_h.append(gcincities_schools_m, ignore_index=True).append(gcincities_schools_e, ignore_index=True).append(gcincities_schools_p, ignore_index=True).append(gcincities_schools_n, ignore_index=True).append(gcincities_schools_d, ignore_index=True)

In [81]:
gcincities_schools.shape

(1470, 7)

In [82]:
gcincities_schools.groupby('City').count()

,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
"Amelia, Clermont County, Ohio",22,22,22,22,22,22
"Blue Ash, Hamilton County, Ohio",80,80,80,80,80,80
"Cincinnati, Ohio",104,104,104,104,104,104
"College Hill, Hamilton County, Ohio",64,64,64,64,64,64
"Delhi Township, Hamilton County, Ohio",46,46,46,46,46,46
"Fairfield, Butler, Ohio",45,45,45,45,45,45
"Forest Park, Hamilton County, Ohio",70,70,70,70,70,70
"Franklin, Warren County, Ohio",30,30,30,30,30,30
"Hamilton, Butler County, OH",36,36,36,36,36,36


##### Like Restaurants, look at all the retrieved categoris and do some grouping/consolidation

In [83]:
gcincities_schools.groupby('Venue Category').count()

,City,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Child Care Service,3,3,3,3,3,3
College Academic Building,4,4,4,4,4,4
College Classroom,2,2,2,2,2,2
Daycare,174,174,174,174,174,174
Elementary School,483,483,483,483,483,483
Football Stadium,1,1,1,1,1,1
High School,365,365,365,365,365,365
Middle School,120,120,120,120,120,120
None,3,3,3,3,3,3


In [84]:
checksch = ['Daycare','Elementary School','Nursery School','Preschool','Elementary School','Middle School','High School']
gcincities_schools_data = gcincities_schools.loc[gcincities_schools['Venue Category'].isin(checksch)]

In [85]:
gcincities_schools_data.groupby('Venue Category').count()

,City,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Daycare,174,174,174,174,174,174
Elementary School,483,483,483,483,483,483
High School,365,365,365,365,365,365
Middle School,120,120,120,120,120,120
Nursery School,109,109,109,109,109,109
Preschool,166,166,166,166,166,166


##### For cleanliness also map Daycare and Nursery Schools as Preschool, for our analysis purpose that is okay

In [86]:
def mapsch(x):
    if x['Venue Category'].find('School') == -1 or x['Venue Category'].find('Nursery School') != -1:
       retval = 'Preschool'
    else:
       retval = x['Venue Category']
   # print(retval)
    return retval

In [87]:
gcincities_schools_data.loc[:,'Venue Category2'] = gcincities_schools_data.apply(mapsch, axis=1)

In [88]:
gcincities_schools_data.groupby('Venue Category2').count()

,City,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
Venue Category2,,,,,,,
Elementary School,483,483,483,483,483,483,483
High School,365,365,365,365,365,365,365
Middle School,120,120,120,120,120,120,120
Preschool,449,449,449,449,449,449,449


In [89]:
gcincities_schools_data.drop(['Venue Category'], inplace=True, axis=1)

In [90]:
gcincities_schools_data.rename(columns={'Venue Category2':'Venue Category'}, inplace=True)

In [91]:
gcincities_schools_data.head()

,City,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,"Cincinnati, Ohio",39.101454,-84.51246,Walnut Hills High School,39.140967,-84.479361,High School
1,"Cincinnati, Ohio",39.101454,-84.51246,School For Creative And Performing Arts,39.107182,-84.517117,High School
2,"Cincinnati, Ohio",39.101454,-84.51246,VLT Academy High School,39.108668,-84.509307,High School
3,"Cincinnati, Ohio",39.101454,-84.51246,Robert A. Taft IT High School,39.109903,-84.523131,High School
4,"Cincinnati, Ohio",39.101454,-84.51246,Cincinnati College Prep Academy,39.110941,-84.527758,High School


#### 3.c) Also get Schools demographic and performance data (grades, population etc.) available at the state education departments sites.

##### This CSV provides the school districts Grade and Performance Index for the last year (2018)

In [92]:
schoolspi = pd.read_csv("Performance_Index_Rankings_CSV.csv")
schoolspi.rename(columns={'LEA IRN':'LEAIRN'}, inplace=True)
schoolspi.sort_values(['LEAIRN'])
schoolspi.set_index('LEAIRN')
schoolspi.head()

,LEA Name,2018 PI Ranking,2018 PI for Ranking,2018 PI Letter Grade,LEAIRN
0,Pathway School of Discovery,665,70.398,D,138
1,Alliance Academy of Cincinnati,785,57.719,F,139
2,Wildwood Environmental Academy,641,72.935,D,222
3,"Ohio Connections Academy, Inc",643,72.869,D,236
4,Summit Academy Community School-Columbus,841,49.245,F,296


##### This CSV provides the school districts headcount of registration in last year (2018), there are various columns for categories of data, our interest may be in the count of #Student_Asian for this particular case

In [93]:
schoolspop = pd.read_csv("oct_hdcnt_fy19_CSV.csv")
schoolspop.rename(columns={'DIST_IRN':'LEAIRN'}, inplace=True)
schoolspop.sort_values(['LEAIRN'])
schoolspop.set_index('LEAIRN')
schoolspop.head()

,LEAIRN,DIST_NAME,COUNTY,ENRL_PRE,ENRL_ELEM,ENRL_MID,ENRL_HIGH,STUDENT_ASIAN,STUDENT_FEMALE,STUDENT_MALE
0,442,Manchester Local,Adams,20,384,202,250,10,410,446
1,43489,Akron City,Summit,553,9816,4534,6373,1807,10343,10952
2,43497,Alliance City,Stark,240,1201,648,954,10,1436,1607
3,43505,Ashland City,Ashland,134,1433,751,872,23,1558,1633
4,43513,Ashtabula Area City,Ashtabula,174,1521,779,945,10,1662,1763


#### Merge the two data

In [94]:
schools_ref = pd.merge(schoolspi,schoolspop, how='right', on=['LEAIRN', 'LEAIRN'])
#schoolspop.join(schoolspi.set_index('LEAIRN'), on='LEAIRN')
schools_ref.head()

,LEA Name,2018 PI Ranking,2018 PI for Ranking,2018 PI Letter Grade,LEAIRN,DIST_NAME,COUNTY,ENRL_PRE,ENRL_ELEM,ENRL_MID,ENRL_HIGH,STUDENT_ASIAN,STUDENT_FEMALE,STUDENT_MALE
0,Manchester Local,489,83.898,D,442,Manchester Local,Adams,20,384,202,250,10,410,446
1,Akron City,691,67.03,D,43489,Akron City,Summit,553,9816,4534,6373,1807,10343,10952
2,Alliance City,637,73.179,D,43497,Alliance City,Stark,240,1201,648,954,10,1436,1607
3,Ashland City,275,90.805,C,43505,Ashland City,Ashland,134,1433,751,872,23,1558,1633
4,Ashtabula Area City,673,69.595,D,43513,Ashtabula Area City,Ashtabula,174,1521,779,945,10,1662,1763


##### Now let us reduce the clutter again, keep only the info needed for our current purpiose keep only the needed columns and give meaningful names

In [95]:
schools_dist = schools_ref[['LEAIRN','LEA Name','2018 PI for Ranking','2018 PI Letter Grade','ENRL_PRE','ENRL_ELEM','ENRL_MID','ENRL_HIGH','STUDENT_ASIAN']].copy()
x0 = {'2018 PI for Ranking':'SD_PI','2018 PI Letter Grade':'SD_Grade'}
schools_dist.rename(columns=x0, inplace=True)

In [96]:
schools_dist.head()

,LEAIRN,LEA Name,SD_PI,SD_Grade,ENRL_PRE,ENRL_ELEM,ENRL_MID,ENRL_HIGH,STUDENT_ASIAN
0,442,Manchester Local,83.898,D,20,384,202,250,10
1,43489,Akron City,67.03,D,553,9816,4534,6373,1807
2,43497,Alliance City,73.179,D,240,1201,648,954,10
3,43505,Ashland City,90.805,C,134,1433,751,872,23
4,43513,Ashtabula Area City,69.595,D,174,1521,779,945,10


 ### ==== 3 Output => we have 3.a) gcincities_schools_data (citywise schools data) 3.b) schools_dist (school district wise Grade/PI and population info) ====

#### 4) Combine all information for final analysis and presentation

#### 4.a) combine city wise schools location data (3a) with city wise restaurants data (2) and group them for city wise representation (one row for each city)

In [97]:
gcincities_all_data = gcincities_venues_data1.append(gcincities_schools_data, ignore_index=True)
gcincities_all_data.groupby('Venue Category').count()

,City,City Latitude,City Longitude,Venue Name,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Chinese Restaurant,475,475,475,475,475,475
Elementary School,483,483,483,483,483,483
High School,365,365,365,365,365,365
Indian Restaurant,215,215,215,215,215,215
Middle School,120,120,120,120,120,120
Other Asian Restaurant,47,47,47,47,47,47
Preschool,449,449,449,449,449,449


##### Transpose the venues using One Hot Encoding to get the venue counts

##### One Hot Encoding

In [98]:
# one hot encoding
gcincities_onehot = pd.get_dummies(gcincities_all_data[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
gcincities_onehot['City'] = gcincities_all_data['City'] 

# move neighborhood column to the first column
fixed_columns = [gcincities_onehot.columns[-1]] + list(gcincities_onehot.columns[:-1])
gcincities_onehot = gcincities_onehot[fixed_columns]

gcincities_onehot.head()

,City,Chinese Restaurant,Elementary School,High School,Indian Restaurant,Middle School,Other Asian Restaurant,Preschool
0,"Cincinnati, Ohio",0,0,0,1,0,0,0
1,"Cincinnati, Ohio",0,0,0,1,0,0,0
2,"Cincinnati, Ohio",0,0,0,1,0,0,0
3,"Cincinnati, Ohio",0,0,0,1,0,0,0
4,"Cincinnati, Ohio",0,0,0,1,0,0,0


In [99]:
gcincities_onehot.shape

(2154, 8)

##### Great. Now make one row for each city, along with count of each venue type  in that city

In [100]:
gcincities_grouped = gcincities_onehot.groupby('City').sum().reset_index()
gcincities_grouped.head()

,City,Chinese Restaurant,Elementary School,High School,Indian Restaurant,Middle School,Other Asian Restaurant,Preschool
0,"Amelia, Clermont County, Ohio",5,7,3,2,3,0,9
1,"Blue Ash, Hamilton County, Ohio",20,23,22,14,6,5,27
2,"Cincinnati, Ohio",37,34,34,17,6,3,23
3,"College Hill, Hamilton County, Ohio",18,25,18,9,5,0,13
4,"Delhi Township, Hamilton County, Ohio",17,14,14,5,4,1,13


##### Now add the LEA numbers from the first df that we read from CSV. We need it so we can join with School District stats. LEA stands for - Local Education Agency

In [101]:
gcincities_grouped_add=gcincities_grouped.join(gcincities.set_index('cityname'), on='City')

In [102]:
gcincities_grouped_add.head()

,City,Chinese Restaurant,Elementary School,High School,Indian Restaurant,Middle School,Other Asian Restaurant,Preschool,population,LEAId,Latitude,Longitude
0,"Amelia, Clermont County, Ohio",5,7,3,2,3,0,9,"4,801",46359,39.028397,-84.217714
1,"Blue Ash, Hamilton County, Ohio",20,23,22,14,6,5,27,"12,114",44867,39.232007,-84.378282
2,"Cincinnati, Ohio",37,34,34,17,6,3,23,"298,843",43752,39.101454,-84.512460
3,"College Hill, Hamilton County, Ohio",18,25,18,9,5,0,13,"14,133",44511,39.202280,-84.547167
4,"Delhi Township, Hamilton County, Ohio",17,14,14,5,4,1,13,"29,510",47373,39.101288,-84.635335


#### 4.b) Then add schools grade and population information (3.b) as well for each city

In [103]:
gcincities_grouped_add2=gcincities_grouped_add.join(schools_dist.set_index('LEAIRN'), on='LEAId')

gcincities_grouped_add2.head()

,City,Chinese Restaurant,Elementary School,High School,Indian Restaurant,Middle School,Other Asian Restaurant,Preschool,population,LEAId,Latitude,Longitude,LEA Name,SD_PI,SD_Grade,ENRL_PRE,ENRL_ELEM,ENRL_MID,ENRL_HIGH,STUDENT_ASIAN
0,"Amelia, Clermont County, Ohio",5,7,3,2,3,0,9,"4,801",46359,39.028397,-84.217714,West Clermont Local,87.406,C,217,3799,1934,2301,167
1,"Blue Ash, Hamilton County, Ohio",20,23,22,14,6,5,27,"12,114",44867,39.232007,-84.378282,Sycamore Community City,103.694,B,91,2501,1240,1640,890
2,"Cincinnati, Ohio",37,34,34,17,6,3,23,"298,843",43752,39.101454,-84.512460,Cincinnati City,72.477,D,1597,16158,7566,9454,498
3,"College Hill, Hamilton County, Ohio",18,25,18,9,5,0,13,"14,133",44511,39.202280,-84.547167,North College Hill City,59.359,F,19,761,437,393,10
4,"Delhi Township, Hamilton County, Ohio",17,14,14,5,4,1,13,"29,510",47373,39.101288,-84.635335,Oak Hills Local,96.997,B,143,3364,1894,2232,90


##### Rename the columns and drop the non useful ones for better readability

In [104]:
x = {'LEAId':'LEAId2','STUDENT_ASIAN':'SD_STUDENT_ASIAN','Elementary School':'SCH_Elementary','High School':'SCH_High',
     'Middle School':'SCH_Middle','Preschool':'SCH_Pre','Indian Restaurant':'REST_Indian','Chinese Restaurant':'REST_Chinese','Other Asian Restaurant':'REST_Asian',
     'LEA Name':'SD_Name','population':'CITY_Population'}
gcincities_grouped_add2.rename(columns=x, inplace=True)

In [105]:
gcincities_grouped_add2.head()

,City,REST_Chinese,SCH_Elementary,SCH_High,REST_Indian,SCH_Middle,REST_Asian,SCH_Pre,CITY_Population,LEAId2,Latitude,Longitude,SD_Name,SD_PI,SD_Grade,ENRL_PRE,ENRL_ELEM,ENRL_MID,ENRL_HIGH,SD_STUDENT_ASIAN
0,"Amelia, Clermont County, Ohio",5,7,3,2,3,0,9,"4,801",46359,39.028397,-84.217714,West Clermont Local,87.406,C,217,3799,1934,2301,167
1,"Blue Ash, Hamilton County, Ohio",20,23,22,14,6,5,27,"12,114",44867,39.232007,-84.378282,Sycamore Community City,103.694,B,91,2501,1240,1640,890
2,"Cincinnati, Ohio",37,34,34,17,6,3,23,"298,843",43752,39.101454,-84.512460,Cincinnati City,72.477,D,1597,16158,7566,9454,498
3,"College Hill, Hamilton County, Ohio",18,25,18,9,5,0,13,"14,133",44511,39.202280,-84.547167,North College Hill City,59.359,F,19,761,437,393,10
4,"Delhi Township, Hamilton County, Ohio",17,14,14,5,4,1,13,"29,510",47373,39.101288,-84.635335,Oak Hills Local,96.997,B,143,3364,1894,2232,90


In [106]:
gcincities_grouped_add2.drop(['LEAId2','Latitude','Longitude'],inplace=True, axis=1)

In [107]:
gcincities_grouped_add2.head()

,City,REST_Chinese,SCH_Elementary,SCH_High,REST_Indian,SCH_Middle,REST_Asian,SCH_Pre,CITY_Population,SD_Name,SD_PI,SD_Grade,ENRL_PRE,ENRL_ELEM,ENRL_MID,ENRL_HIGH,SD_STUDENT_ASIAN
0,"Amelia, Clermont County, Ohio",5,7,3,2,3,0,9,"4,801",West Clermont Local,87.406,C,217,3799,1934,2301,167
1,"Blue Ash, Hamilton County, Ohio",20,23,22,14,6,5,27,"12,114",Sycamore Community City,103.694,B,91,2501,1240,1640,890
2,"Cincinnati, Ohio",37,34,34,17,6,3,23,"298,843",Cincinnati City,72.477,D,1597,16158,7566,9454,498
3,"College Hill, Hamilton County, Ohio",18,25,18,9,5,0,13,"14,133",North College Hill City,59.359,F,19,761,437,393,10
4,"Delhi Township, Hamilton County, Ohio",17,14,14,5,4,1,13,"29,510",Oak Hills Local,96.997,B,143,3364,1894,2232,90


##### Lastly reseuqence the columns and drop the non useful ones for better readability

In [108]:
cols = ['City','CITY_Population', 'REST_Chinese', 'REST_Indian', 'REST_Asian', 'SCH_Pre', 'SCH_Elementary', 'SCH_Middle', 'SCH_High', 'SD_Name', 'SD_Grade', 'SD_PI','ENRL_PRE','ENRL_ELEM','ENRL_MID','ENRL_HIGH', 'SD_STUDENT_ASIAN']
gcincities_grouped_final = gcincities_grouped_add2[cols]

In [109]:
print('So finally we have got all the information that our client needs for decision making for the {} cities in the Greater Cincinnati Area.'.format(gcincities_grouped_final.shape[0]))

So finally we have got all the information that our client needs for decision making for the 27 cities in the Greater Cincinnati Area.


In [110]:
gcincities.to_csv('1gcincities.csv')
gcincities_venues_data1.to_csv('2gcincities_venues_data1.csv')
gcincities_schools_data.to_csv('3gcincities_schools_data.csv')
schools_dist.to_csv('4schools_dist.csv')
gcincities_grouped_final.to_csv('5gcincities_grouped_final.csv')
gcincities_rem.to_csv('9gcincities_rem.csv')

In [111]:
final_data_rep=gcincities_grouped_final

### 5) Final Analysis and Results

##### So here our final data is ready and presentable. This can be
##### - manually read for human consumption and analysis,
##### - represented using visualization techniques,
##### - clustered/classified based on specific needs.

##### For our requirement that is to *"find the best market to launch the new Asian Grocery Supermarket"*, let us use some technique to rank the cities. 
##### We will assign scores (Index) for each information criteria gathered, and total that to come with the final index for this city. For simplification, let us use just the total value of each criteria (Restaurants Total, Schools Total, Enrollments Total).

##### First Get the Totals and drop individual columns

In [112]:

final_data_rep.loc[:,'REST_Tot'] = final_data_rep.loc[:,'REST_Chinese'] + final_data_rep.loc[:,'REST_Indian'] + final_data_rep.loc[:,'REST_Asian']
final_data_rep.drop(['REST_Chinese','REST_Indian','REST_Asian'], inplace=True, axis=1)
final_data_rep.loc[:,'SCH_Tot'] = final_data_rep.loc[:,'SCH_Pre'] + final_data_rep.loc[:,'SCH_Elementary'] + final_data_rep.loc[:,'SCH_Middle']+final_data_rep.loc[:,'SCH_High']
final_data_rep.drop(['SCH_Pre','SCH_Elementary','SCH_Middle','SCH_High'], inplace=True, axis=1)
final_data_rep.loc[:,'ENRL_Tot'] = final_data_rep.loc[:,'ENRL_PRE'] + final_data_rep.loc[:,'ENRL_ELEM'] + final_data_rep.loc[:,'ENRL_MID']+final_data_rep.loc[:,'ENRL_HIGH']
final_data_rep.drop(['ENRL_PRE','ENRL_ELEM','ENRL_MID','ENRL_HIGH'], inplace=True, axis=1)
final_data_rep.drop(['SD_Name','SD_PI'], inplace=True, axis=1)
final_data_rep.head()

,City,CITY_Population,SD_Grade,SD_STUDENT_ASIAN,REST_Tot,SCH_Tot,ENRL_Tot
0,"Amelia, Clermont County, Ohio","4,801",C,167,7,22,8251
1,"Blue Ash, Hamilton County, Ohio","12,114",B,890,39,78,5472
2,"Cincinnati, Ohio","298,843",D,498,57,97,34775
3,"College Hill, Hamilton County, Ohio","14,133",F,10,27,61,1610
4,"Delhi Township, Hamilton County, Ohio","29,510",B,90,23,45,7633


##### Now the below function to assing scores to cities for individual criteria
##### NOTE: This exercise is done in simplest possible way for this work, it can be made more sophisticated criteria and scoring mechanism as required.

In [113]:
def map_rest(x):
    if x['REST_Tot'] > 50:
       retval = 8
    elif x['REST_Tot'] > 30:
       retval = 6
    elif x['REST_Tot'] >10:
       retval = 4
    else:
       retval = 2
    return retval

def map_sch(x):
    if x['SCH_Tot'] > 50:
       retval = 4
    elif x['SCH_Tot'] > 35:
       retval = 3
    elif x['SCH_Tot'] >20:
       retval = 2
    else:
       retval = 1
    return retval

def map_sd(x):
    if x['SD_Grade'] == 'A':
       retval = 4
    elif x['SD_Grade'] == 'B':
       retval = 3
    elif x['SD_Grade'] == 'C':
       retval = 1
    else:
       retval = 1
    return retval

def map_ast(x):
    if x['SD_STUDENT_ASIAN'] > 500:
       retval = 4
    elif x['SD_STUDENT_ASIAN'] > 200:
       retval = 3
    elif x['SD_STUDENT_ASIAN'] > 100:
       retval = 1
    else:
       retval = 0
    return retval

def map_enr(x):
    if x['ENRL_Tot'] > 10000:
       retval = 4
    elif x['ENRL_Tot'] > 5000:
       retval = 3
    elif x['ENRL_Tot'] > 2000:
       retval = 1
    else:
       retval = 1
    return retval

In [114]:
final_data_rep.loc[:,'REST_IND'] = final_data_rep.apply (lambda row: map_rest(row),axis=1)
final_data_rep.loc[:,'SCH_IND'] = final_data_rep.apply (lambda row: map_sch(row),axis=1)
final_data_rep.loc[:,'SD_IND'] = final_data_rep.apply (lambda row: map_sd(row),axis=1)
final_data_rep.loc[:,'AST_IND'] = final_data_rep.apply (lambda row: map_ast(row),axis=1)
final_data_rep.loc[:,'ENR_IND'] = final_data_rep.apply (lambda row: map_enr(row),axis=1)
final_data_rep.head()

,City,CITY_Population,SD_Grade,SD_STUDENT_ASIAN,REST_Tot,SCH_Tot,ENRL_Tot,REST_IND,SCH_IND,SD_IND,AST_IND,ENR_IND
0,"Amelia, Clermont County, Ohio","4,801",C,167,7,22,8251,2,2,1,1,3
1,"Blue Ash, Hamilton County, Ohio","12,114",B,890,39,78,5472,6,4,3,4,3
2,"Cincinnati, Ohio","298,843",D,498,57,97,34775,8,4,1,3,4
3,"College Hill, Hamilton County, Ohio","14,133",F,10,27,61,1610,4,4,1,0,1
4,"Delhi Township, Hamilton County, Ohio","29,510",B,90,23,45,7633,4,3,3,0,3


##### Sum up the individual scores for each city into Final score FINAL_IND and rank accordingly

In [115]:
final_data_rep.loc[:,'FINAL_IND'] = final_data_rep.loc[:,'REST_IND'] + final_data_rep.loc[:,'SCH_IND'] + final_data_rep.loc[:,'AST_IND'] + final_data_rep.loc[:,'ENR_IND'] + final_data_rep.loc[:,'SD_IND']
final_data_rep.sort_values('FINAL_IND',ascending=False)

,City,CITY_Population,SD_Grade,SD_STUDENT_ASIAN,REST_Tot,SCH_Tot,ENRL_Tot,REST_IND,SCH_IND,SD_IND,AST_IND,ENR_IND,FINAL_IND
12,"Mason, Warren County, Ohio","30,712",B,2820,35,58,10396,6,4,3,4,4,21
2,"Cincinnati, Ohio","298,843",D,498,57,97,34775,8,4,1,3,4,20
1,"Blue Ash, Hamilton County, Ohio","12,114",B,890,39,78,5472,6,4,3,4,3,20
26,"Westwood, Hamilton County, Ohio","29,950",D,498,37,52,34775,6,4,1,3,4,18
10,"Liberty Township, Butler County, Ohio,37","259""",B,1188,21,40,16262,4,3,3,4,4,18
22,"Springdale, Hamilton County, Ohio","11,223",D,209,37,63,5955,6,4,1,3,3,17
19,"Sharonville, Hamilton County, Ohio","13,560",D,209,48,71,5955,6,4,1,3,3,17
9,"Indian Hill, Ohio","5,785",A,215,30,58,2030,4,4,4,3,1,16
5,"Fairfield, Butler, Ohio","42,510",C,561,14,44,9887,4,3,1,4,3,15
23,"St. Bernard, Hamilton County, Ohio","4,368",D,10,60,105,905,8,4,1,0,1,14


#### Now it will be best to see the numbers in a visual form to help decide better.
##### Let us get the location of each city and view them on map using FOLIUM
##### Woudn't it be awesome to see the city's circle mark sized based on its final score? so we do not have to go back and forth to see final index and location of each city?

In [116]:
final_city_index_vis=gcincities_retained.join(final_data_rep.set_index('City'), on='cityname')

final_city_index_vis.head()

,cityname,population,LEAId,Latitude,Longitude,CITY_Population,SD_Grade,SD_STUDENT_ASIAN,REST_Tot,SCH_Tot,ENRL_Tot,REST_IND,SCH_IND,SD_IND,AST_IND,ENR_IND,FINAL_IND
0,"Cincinnati, Ohio","298,843",43752,39.101454,-84.512460,"298,843",D,498,57,97,34775,8,4,1,3,4,20
1,"Hamilton, Butler County, OH","62,447",44107,39.399507,-84.561343,"62,447",D,56,10,35,10067,2,2,1,0,4,9
2,"Liberty Township, Butler County, Ohio,37","259""",46110,39.401558,-84.409678,"259""",B,1188,21,40,16262,4,3,3,4,4,18
3,"Middletown, Butler County, Ohio","48,694",44404,39.515058,-84.398276,"48,694",D,15,6,24,6338,2,2,1,0,3,8
4,"Fairfield, Butler, Ohio","42,510",46102,39.345895,-84.560503,"42,510",C,561,14,44,9887,4,3,1,4,3,15


In [117]:
address = 'Cincinnati, Ohio'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Cincinnati are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Cincinnati are 39.1014537, -84.5124602.


In [118]:
# create map of New York using latitude and longitude values
map_gtcincinnati2 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city, rad in zip(final_city_index_vis['Latitude'], final_city_index_vis['Longitude'], final_city_index_vis['cityname'],final_city_index_vis['FINAL_IND']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=rad,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_gtcincinnati2)  
    
map_gtcincinnati2

#### Here we go! All cities sized based on thier final score. Makes it so each to see that the cities like **Mason, OH, Blue Ash, OH, Cincinnati, OH and Westwood, OH** and most potential candidates for the new market.
#### While cities like **Oxford, Franklin, Amelia, Hamilton, South Lebanon** can be ruled out for their lack of Asian Index and distance from other cities with good ranks.
#### Our client will use this information and do other checks like (if other existing such supermarkets, their size/reviews etc.) to take the final decision.

## Conclude:

#### So we helped out client choose the best fit city to launch the new Asian Grocery Supermarket. We achieved this by collecting, cleansing, preparing, analyzing, summarizing, and presenting the data in the best way so he can visualize the cities and their information.

#### Additional Notes:
##### The above work only shows basic work done, this analysis can be further extended/modified to analyze cities for various other clients and their specific asks.
#### This work and the data can be extended and can be used in many other ways like
##### - manually read for human consumption and analysis,
##### - represented using visualization techniques,
##### - clustered/classified based on specific needs.


##### For Ex> 
##### i) An individual customer who wants to relocate from some other city of US to greater Cincinnati can use us to know which city is best to rent/buy a home for the family. We can analyze his original city and find a city with similar characteristics from the list that we gathered within Cincinnati area using clustering techniques.
##### ii) Any other new business ventures that wants to come to Cincinnati can use this information for demographic analysis of cities. like to host Korean Movies in theaters, open Italian Restaurant etc.
##### And of course, this entire thing can be easily replicated for any other city, metropolitan area across the globe with little tweaking!